# Darts

In [6]:
pip install darts

     -------------------------------------- 589.7/589.7 kB 2.9 MB/s eta 0:00:00
     ---------------------------------------- 73.9/73.9 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 435.6/435.6 kB 1.6 MB/s eta 0:00:00
     -------------------------------------- 800.3/800.3 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 43.8/43.8 kB ? eta 0:00:00
     -------------------------------------- 147.7/147.7 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 89.1/89.1 MB 3.7 MB/s eta 0:00:00
     -------------------------------------- 162.6/162.6 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 92.0/92.0 kB 2.6 MB/s eta 0:00:00
     ---------------------------------------- 1.0/1.0 MB 3.6 MB/s eta 0:00:00
     -------------------------------------- 970.0/970.0 kB 4.1 MB/s eta 0:00:00
     ----------------------

In [7]:
import pandas as pd
from darts import TimeSeries

In [15]:
# import csv dataset
df=pd.read_csv('divvy-tripdata_cleaned.csv',delimiter=",")

In [18]:
series = TimeSeries.from_dataframe(df, "start_time", "trip_duration", fill_missing_dates=True)

ValueError: Could not find a unique inferred frequency (not constant). Observed frequencies: {'29T', '36T', '11T', '15T', '2T', '53T', '14T', '8T', '9T', '16T', '35T', '13T', '3T', '41T', '19T', 'T', '5T', '25T', '45T', '18T', '6T', '12T', '20T', '10T', '24T', '21T', '17T', '48T', '4T', '28T', '47T', '7T', '23T', '22T'}. If any of those is the actual frequency, try passing it with fill_missing_dates=True and freq=your_frequency. For more information about frequency aliases, read https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases


ValueError: Could not find a unique inferred frequency (not constant). Observed frequencies: {'29T', '36T', '11T', '15T', '2T', '53T', '14T', '8T', '9T', '16T', '35T', '13T', '3T', '41T', '19T', 'T', '5T', '25T', '45T', '18T', '6T', '12T', '20T', '10T', '24T', '21T', '17T', '48T', '4T', '28T', '47T', '7T', '23T', '22T'}. If any of those is the actual frequency, try passing it with fill_missing_dates=True and freq=your_frequency. For more information about frequency aliases, read https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases